# This notebook is useful to find the best parameters for the RNN models

In [35]:
#Imports


import os
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from tensorflow import keras
from keras import layers

from sklearn.preprocessing import MinMaxScaler
import sklearn.preprocessing as sk

import helper_functions as hf

from analysis_tools import *

CURDIR = os.getcwd()
DATADIR = os.path.join(CURDIR,  "data")
FIGDIR = os.path.join(CURDIR,  "figure")

In [36]:

list_of_parameters = {
'type_of_data' : ['Price', 'All'],
'add_return' : [False, True],
'add_log_functions' : [False,True],
'drop_higly_correlated_data' : [True, False],
'add_analysed_curves' : [True, False],
'add_return' : [True, False],
'scaler' : [sk.MinMaxScaler],
'feature_range' :  [(0,1), (-1,1)],

'rnn_size' : [10,1,7, 8, 9, 11, 12],
'size_train' : [2000],
'size_valid' : [800],
'target_id' : [0],
'type_of_rnn' : ['GRU', 'LSTM'],
'nb_layers' : [1],
'nb_units' : [[128],[256]],
'learning_rate' : [0.001],
'optimizer' : [keras.optimizers.Adam],
'loss' : [keras.losses.MeanSquaredError()],
'batch_size' : [4,8,16],
'epochs' : [120]}

In [37]:
params = {
    'type_of_data' : 'Price',
    'add_log_functions' : True,
    'drop_higly_correlated_data' : False,
    'add_analysed_curves' :False,
    'add_return' : False,
    'scaler' :sk.MinMaxScaler,
    'feature_range': (0,1),

    'rnn_size' :1,
    'size_train' : 2000,
    'size_valid' : 800,
    'target_id' : 0,
    'type_of_rnn' : 'GRU',
    'nb_layers' : 1,
    'nb_units' : [128],
    'learning_rate' : 0.001,
    'optimizer' : keras.optimizers.Adam,
    'loss' : keras.losses.MeanSquaredError(),
    'batch_size' : 16,
    'epochs' : 180

}




In [38]:
def evaluate(params): # takes as agument the parameters of the model and returns the rmse of this model
    
    type_of_data = params['type_of_data']
    add_log_functions = params['add_log_functions']
    drop_higly_correlated_data = params['drop_higly_correlated_data']
    add_analysed_curves = params['add_analysed_curves']
    add_return = params['add_return']
    scaler, feature_range = params['scaler'], params['feature_range']

    # params model rnn
    rnn_size = params['rnn_size']
    size_train = params['size_train']
    size_valid = params['size_valid']
    size_test = 3544 - size_train - size_valid
    target_id = params['target_id']
    type_of_rnn = params['type_of_rnn']
    nb_layers = params['nb_layers']
    nb_units = params['nb_units']
    learning_rate = params['learning_rate']
    optimizer = params['optimizer']
    loss = params['loss']
    batch_size = params['batch_size']
    epochs = params['epochs']

    df_blockchain = pd.read_csv(os.path.join(DATADIR, "df_blockchain.csv"), 
                            delimiter=",")
    # nettoyer les datas
    index_with_nan = df_blockchain.index[df_blockchain.isnull().any(axis=1)]
    df_blockchain.drop(index_with_nan,0, inplace=True)
    if add_log_functions:
        df_blockchain['market-price'] = np.log(df_blockchain['market-price'])
    if type_of_data == 'Price':
        if add_analysed_curves:
            df = df_blockchain["market-price"]
            ma1_2 = deltamma(df, 20, 50)
            macd1, macd2, macd3 = macd(df, .75, 12, 26)
            boll = bollinger(df, 20, 2)
            new_val = []
            for i in [ma1_2, macd1, macd3, boll]:
                for j in i.columns:
                    new_val.append(j)
            macd1 = macd1[50 - 26:]
            macd2 = macd2[50 - 26 - 9:]
            macd3 = macd3[50 - 26 - 9:]
            boll = boll[50 - 20:]
            df_blockchain = df_blockchain[50:]
            macd1.reset_index(inplace=True)
            macd2.reset_index(inplace=True)
            macd3.reset_index(inplace=True)
            boll.reset_index(inplace=True)
            df_blockchain.reset_index(inplace=True)
            df_blockchain = pd.concat([df_blockchain, ma1_2, macd1, macd3, boll], axis=1)

            columns = ["market-price"]+new_val
            dataset = df_blockchain[columns]
        else:
            columns = ['market-price']
            dataset = df_blockchain[columns]

    if type_of_data == 'All':
        if add_analysed_curves:
            df = df_blockchain['market-price']
            ma1_2 = deltamma(df, 20, 50)
            macd1, macd2, macd3 = macd(df, .75, 12, 26)
            boll = bollinger(df, 20, 2)
            new_val = []
            for i in [ma1_2, macd1, macd3, boll]:
                for j in i.columns:
                    new_val.append(j)
            macd1 = macd1[50 - 26:]
            macd2 = macd2[50 - 26 - 9:]
            macd3 = macd3[50 - 26 - 9:]
            boll = boll[50 - 20:]
            df_blockchain = df_blockchain[50:]
            macd1.reset_index(inplace=True)
            macd2.reset_index(inplace=True)
            macd3.reset_index(inplace=True)
            boll.reset_index(inplace=True)
            df_blockchain.reset_index(inplace=True)
            df_blockchain = pd.concat([df_blockchain, ma1_2, macd1, macd3, boll], axis=1)

            if drop_higly_correlated_data:
                columns = ["market-price","n-transactions-per-block","hash-rate","difficulty","miners-revenue","trade-volume","blocks-size","avg-block-size","transaction-fees","transaction-fees-usd","cost-per-transaction-percent","cost-per-transaction","n-transactions","n-transactions-total","n-transactions-excluding-popular","estimated-transaction-volume-usd","total-bitcoins","market-cap"]
                columns = columns + new_val
                dataset = df_blockchain[columns]   
            else:
                columns = list(df_blockchain.columns)[1:] + new_val
                dataset = df_blockchain[columns]
        else:
            if drop_higly_correlated_data:
                columns = ["market-price","n-transactions-per-block","hash-rate","difficulty","miners-revenue","trade-volume","blocks-size","avg-block-size","transaction-fees","transaction-fees-usd","cost-per-transaction-percent","cost-per-transaction","n-transactions","n-transactions-total","n-transactions-excluding-popular","estimated-transaction-volume-usd","total-bitcoins","market-cap"]
                dataset = df_blockchain[columns]   
            else:
                columns = list(df_blockchain.columns)[1:]
                dataset = df_blockchain[columns]
    if add_return:
        col = dataset['market-price'].diff().iloc[1:]
        col[0] = 0
        dataset['ret'] = col
        columns.append('ret')

    try:
        dataset.drop(columns='index', inplace=True)
    except:
        pass
    try:
        dataset.drop(columns='Date', inplace=True)
    except:
        pass

    
    for i,x in enumerate(columns):
        if x == 'index' or x == 'Date':
            columns.pop(i)

    # index_with_nan = dataset.index[dataset.iloc[:,1].isnull()]
    # dataset.drop(index_with_nan,0, inplace=True)



    scaler = scaler(feature_range=feature_range)
    dataset = scaler.fit_transform(dataset.values.reshape(-1,len(columns)))
    data_train = dataset[:size_train]
    data_valid = dataset[size_train:size_train+size_valid]
    data_test = dataset[size_train+size_valid:size_train+size_valid+size_test]
    def process_data(data, rnn_size=rnn_size, target_id=target_id, columns_size=len(columns)):
        X = []
        y = []
        for i in range(len(data)-rnn_size):
            X.append(data[i:i+rnn_size,:])
            y.append(data[i+rnn_size,target_id])
        return np.array(X).astype(np.float32).reshape((-1,rnn_size,columns_size)), np.array(y).astype(np.float32)

    # process data for RNN
    X_train, y_train = process_data(data_train, rnn_size)
    X_val, y_val = process_data(data_valid, rnn_size)
    X_test, y_test = process_data(data_test, rnn_size)

    callback= keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0,
    patience=15,
    verbose=0,
    mode="auto",
    restore_best_weights = True)

    if type_of_rnn == 'LSTM':
        regressor = keras.Sequential()
        regressor.add(
            keras.layers.Bidirectional(
                keras.layers.LSTM(
                    units = nb_units[0],
                    input_shape = (X_train.shape[1],X_train.shape[2] )
                )
            )
        )
        for i in range(1,nb_layers):
            regressor.add(keras.layers.Dense(units = nb_units[i]))
        regressor.add(keras.layers.Dense(units = 1))
        
        opt = optimizer(learning_rate=learning_rate)
        #Compiling the Recurrent Neural Network with adam optimier and 'mean_absolute_error' as loss function
        regressor.compile(loss=loss, optimizer=opt)
    else:
        regressor = keras.Sequential()
        regressor.add(
            keras.layers.Bidirectional(
                keras.layers.GRU(
                    units = nb_units[0],
                    input_shape = (X_train.shape[1],X_train.shape[2] )
                )
            )
        )
        for i in range(1,nb_layers):
            regressor.add(keras.layers.Dense(units = nb_units[i]))
        regressor.add(keras.layers.Dense(units = 1))
        
        opt = optimizer(learning_rate=learning_rate)
        #Compiling the Recurrent Neural Network with adam optimier and 'mean_absolute_error' as loss function
        regressor.compile(loss=loss, optimizer=opt)
    history = regressor.fit(X_train, y_train, validation_data = (X_val, y_val), batch_size = batch_size, epochs = epochs, callbacks=[callback])
    y_pred = regressor.predict(X_test)
    # compute rmse for test
    y_pred_inverse = scaler.inverse_transform(np.concatenate([y_pred, data_test[-len(y_pred):,1:]], axis=1))
    y_test_inverse = scaler.inverse_transform(data_test.reshape(-1,len(columns)))[rnn_size:]
    if add_log_functions:
        y_pred_inverse = np.exp(y_pred_inverse)
        y_test_inverse = np.exp(y_test_inverse)
    rmse_score = np.sqrt(np.square(np.subtract(y_pred_inverse, y_test_inverse)).mean())
    return rmse_score

# Méthode par tatonnement, on cherche les meilleurs paramètres de façon 'indépendante'

In [39]:
#On fait varier le rnn size

# values = []

# for size in range(1,20):
#     print(size)
#     params['rnn_size'] = size
#     tot = 0
#     nb_test = 10
#     for j in range(nb_test):
#         tot+=evaluate(params)
#     values.append(tot/nb_test)
    
# min en 1

In [40]:
# # On fait varier le nombre de couches

# values = []
# for i in [1,2]:
#     tot = 0
#     params['nb_layers']= i
#     if i == 1:
#         params['nb_units'] = [128]
#     else:
#         params['nb_units'] = [128, 64]
#     n_test = 5
#     for j in range(n_test):
#         print('J en suis a ('+str(i)+','+str(j)+')')
#         tot+= evaluate(params)
#     values.append(tot/n_test)

# # on trouve un meilleur résultat avec une seule couche

In [41]:
# #on utilise le log price ou non 

# values = []
# for bo in [True,False]:
#     params['add_log_functions'] = bo
#     tot  =0
#     n_test = 5
#     for j in range(n_test):
#         print('J en suis a ('+str(bo)+','+str(j)+')')

#         tot+= evaluate(params)
#     values.append(tot/n_test)
# print(values)

# #C'est mieux de ne pas passer au log 

In [42]:
# # on change le nombre de neurones 

# values = []
# for nb in [64, 128,256]:
#     params['nb_units'] = [nb]
#     tot  =0
#     n_test = 5
#     for j in range(n_test):
#         print('J en suis a ('+str(nb)+','+str(j)+')')

#         tot+= evaluate(params)
#     values.append(tot/n_test)
# print(values)

# valeurs très similaires mais batch size de 128 légèrement meilleur

In [43]:
# # on fait varier la taille du batch size 

# values = []
# for nb in [2,4]:
#     params['batch_size'] = nb
#     tot  =0
#     n_test = 5
#     for j in range(n_test):
#         print('J en suis a ('+str(nb)+','+str(j)+')')

#         tot+= evaluate(params)
#     values.append(tot/n_test)
# print(values)



# Choix aléatoire de paramètres pour trouver un minimum global de la rmse

In [44]:
liste_params_chosen = []
N = 2 # Nombre d'ensembles de paramètres 
for i in range(N):
    set = {}
    for par in list_of_parameters:
        entier = np.random.randint(0,len(list_of_parameters[par]))
        x = list_of_parameters[par][entier]
        params[par] = x
        set[par] = x
    liste_params_chosen.append(set)


In [45]:
search_best_params = {}
for i,par in enumerate(liste_params_chosen):
    search_best_params[i] = (par,evaluate(par))
print(search_best_params)
l = []
l.extend(search_best_params.values())
# print(l[0])
l1 = []
for j in range(N):
    l1.append(l[j][1])
print(l1)

Epoch 1/120


<ipython-input-38-82442111bbda>:29: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_blockchain.drop(index_with_nan,0, inplace=True)


125/125 [==============================] - 5s 7ms/step - loss: 0.0156 - val_loss: 0.0019
Epoch 2/120
125/125 [==============================] - 0s 4ms/step - loss: 6.9453e-05 - val_loss: 3.8606e-05
Epoch 3/120
125/125 [==============================] - 0s 3ms/step - loss: 2.0258e-05 - val_loss: 3.4289e-05
Epoch 4/120
125/125 [==============================] - 0s 3ms/step - loss: 2.0311e-05 - val_loss: 3.0738e-05
Epoch 5/120
125/125 [==============================] - 0s 3ms/step - loss: 2.0663e-05 - val_loss: 2.1858e-05
Epoch 6/120
125/125 [==============================] - 0s 3ms/step - loss: 2.1167e-05 - val_loss: 2.1447e-05
Epoch 7/120
125/125 [==============================] - 0s 3ms/step - loss: 2.0750e-05 - val_loss: 2.3549e-05
Epoch 8/120
125/125 [==============================] - 0s 3ms/step - loss: 2.1666e-05 - val_loss: 5.5303e-05
Epoch 9/120
125/125 [==============================] - 0s 3ms/step - loss: 2.0931e-05 - val_loss: 2.7667e-05
Epoch 10/120
125/125 [=================

<ipython-input-38-82442111bbda>:29: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_blockchain.drop(index_with_nan,0, inplace=True)


Epoch 1/120
125/125 [==============================] - 5s 17ms/step - loss: 8.4010e-04 - val_loss: 0.0018
Epoch 2/120
125/125 [==============================] - 1s 11ms/step - loss: 2.4243e-06 - val_loss: 4.4394e-04
Epoch 3/120
125/125 [==============================] - 2s 12ms/step - loss: 1.3205e-06 - val_loss: 4.4131e-04
Epoch 4/120
125/125 [==============================] - 2s 12ms/step - loss: 1.2836e-06 - val_loss: 4.1160e-04
Epoch 5/120
125/125 [==============================] - 2s 13ms/step - loss: 1.2654e-06 - val_loss: 3.9135e-04
Epoch 6/120
125/125 [==============================] - 2s 13ms/step - loss: 1.1577e-06 - val_loss: 3.7195e-04
Epoch 7/120
125/125 [==============================] - 2s 13ms/step - loss: 1.0645e-06 - val_loss: 3.5468e-04
Epoch 8/120
125/125 [==============================] - 2s 12ms/step - loss: 1.1498e-06 - val_loss: 3.3477e-04
Epoch 9/120
125/125 [==============================] - 2s 13ms/step - loss: 1.0884e-06 - val_loss: 3.1281e-04
Epoch 10/120
1